In [ ]:
!pip install geopandas metpy xarray[complete] -q
!pip install eccodes cfgrib rioxarray ecmwf.opendata 

In [2]:
import xarray as xr
import cfgrib
import os 
from ecmwf.opendata import Client
# plt.style.available
#import colormaps as cmaps 
import concurrent
import numpy as np


from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()
client = Client(source="ecmwf")
import metpy.calc as mpcalc
from metpy.units import units
import json
import pandas as pd
from datetime import datetime, timedelta

import rioxarray
import xarray as xr
import cfgrib

import json
import zipfile
import os
from os import walk
import netCDF4 as nc
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import shapely
from shapely.geometry import Point
import xarray as xr
import logging

In [3]:

def dict_l(data, fm):
    try:
        # Validate filename
       
        
        # Convert data to dictionary
        logging.info("Converting data to dictionary...")
        data_dict = data.squeeze().to_dict(data='list', encoding=False)
        
        # Helper function to recursively round numeric values
        def round_numbers(obj):
            if isinstance(obj, dict):
                return {k: round_numbers(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [round_numbers(item) for item in obj]
            elif isinstance(obj, (float, np.float32, np.float64)):
                return round(float(obj), 2)  # Round floats to 2 decimal places
            elif isinstance(obj, (int, np.int32, np.int64)):
                return int(obj)  # Keep integers as-is
            else:
                return obj  # Leave other types unchanged
        
        # Apply rounding to the dictionary
        logging.info("Rounding numeric values to 2 decimal places...")
        rounded_data_dict = round_numbers(data_dict)
        
        # Helper function to handle non-serializable types
        def convert_types(obj):
            if isinstance(obj, dict):
                return {k: convert_types(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_types(item) for item in obj]
            elif isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
                return obj.isoformat()
            elif isinstance(obj, timedelta):
                return obj.total_seconds()
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.float32, np.float64)):
                return float(obj)
            elif isinstance(obj, (np.int32, np.int64)):
                return int(obj)
            raise TypeError(f"Type {type(obj)} not serializable")
        
        # Save to JSON file
        file = os.path.join(os.getcwd(), f"{fm}.json")
        logging.info(f"Saving data to {file}...")
        with open(file, 'w') as json_file:
            json.dump(rounded_data_dict, json_file, indent=4, default=convert_types)
        
        logging.info("Data successfully saved.")
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [ ]:
# HRES:

# fc: Forecast.
# ENS:

# cf: Control forecast.
# pf: Perturbed forecast.
# em: Ensemble mean.
# es: Ensemble standard deviation.
# ep: Probabilities.
# Valid values for stream are:

# oper: Atmospheric fields from HRES - 00 UTC and 12 UTC.
# wave: Ocean wave fields from HRES - 00 UTC and 12 UTC.
# enfo: Atmospheric fields from ENS.
# waef: Ocean wave fields from ENS.
# scda: Atmospheric fields from HRES - 06 UTC and 18 UTC.
# scwv: Ocean wave fields from HRES - 06 UTC and 18 UTC.

In [5]:
step = list(range(3, 73, 3))

client.retrieve(
    time=0,
    type="fc",
    stream='oper',step=step,
    param=['d','gh',"r","vo"],
    levelist=[ 850, 500],
    target="atms.grib2",
)

<multiple>:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [6]:
ds_850=xr.open_dataset('atms.grib2',engine='cfgrib').sel(longitude=slice(60,100),latitude=slice(40,0),isobaricInhPa=850)
ds_500=xr.open_dataset('atms.grib2',engine='cfgrib').sel(longitude=slice(60,100),latitude=slice(40,0),isobaricInhPa=500)


In [9]:
gh8=ds_850['gh']
d8=ds_850['d']
r8=ds_850['r']
vo8=ds_850['vo']

dict_l(gh8,"gh850")
dict_l(d8*1e5,"div850")
dict_l(r8,"rh850")
dict_l(vo8*1e5,"vo850")




gh5=ds_500['gh']
d5=ds_500['d']
r5=ds_500['r']
vo5=ds_500['vo']

dict_l(gh5,"gh500")
dict_l(d5*1e5,"div500")
dict_l(r5,"rh500")
dict_l(vo5*1e5,"vo500")

ds_850.close()
ds_500.close()


In [10]:
step = list(range(3, 73, 3))


client.retrieve(
    time=0,
    type="fc",
    stream='oper',step=step,
    param=['tp','tcwv',"msl","2t"],
    target="atms.grib2",
)




<multiple>:   0%|          | 0.00/60.7M [00:00<?, ?B/s]

In [15]:
# import cfgrib
t2 = xr.open_dataset('atms.grib2',engine='cfgrib',decode_coords="all",
                     backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 't2m',}}).sel(longitude=slice(60,100),latitude=slice(40,0))['t2m']
ds=xr.open_dataset('atms.grib2',engine='cfgrib')
rf = ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tp'][:, :, :]  # Select a specific time slice
p =ds.sel(longitude=slice(60,100),latitude=slice(40,0))['msl'][:, :, :]/100  # Select a specific time slice
tcw=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tcwv']

rf2=rf* units('m').to(units.mm)


In [17]:
dict_l(t2,"2m_temp")
dict_l(rf2,"rf")
dict_l(p,"msl")
dict_l(tcw,"tcwv")
# dict(w10,"w10")
# dict(tcw,"tcwv")
